### first attempt with incomplete dataset
- unusable due to low performance

In [11]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from classes import Document, Annotation

In [12]:
import spacy
from spacy import displacy
from spacy.tokens import DocBin

In [13]:
data = pd.read_json('labels_new.json')
data.head()

,documents
0,"{'annotations': [{'end': 18, 'id': 1, 'label':..."
1,"{'annotations': [{'end': 16, 'id': 1, 'label':..."
2,"{'annotations': [{'end': 22, 'id': 1, 'label':..."
3,"{'annotations': [{'end': 36, 'id': 1, 'label':..."
4,"{'annotations': [{'end': 87, 'id': 1, 'label':..."


In [4]:
data['documents'][0]['annotations']

[{'end': 13, 'id': 1, 'label': 'jobTitle', 'start': 3},
 {'end': 31, 'id': 2, 'label': 'jobSkill', 'start': 13},
 {'end': 501, 'id': 3, 'label': 'jobSkill', 'start': 494},
 {'end': 1023, 'id': 4, 'label': 'jobSkill', 'start': 1014},
 {'end': 1110, 'id': 5, 'label': 'jobSkill', 'start': 1084},
 {'end': 1136, 'id': 6, 'label': 'jobSkill', 'start': 1131},
 {'end': 1334, 'id': 7, 'label': 'jobTitle', 'start': 1317},
 {'end': 1542, 'id': 8, 'label': 'softSkill', 'start': 1530},
 {'end': 3350, 'id': 9, 'label': 'jobTitle', 'start': 3325},
 {'end': 5508, 'id': 10, 'label': 'jobSkill', 'start': 5498},
 {'end': 5536, 'id': 11, 'label': 'jobSkill', 'start': 5528},
 {'end': 5540, 'id': 12, 'label': 'jobSkill', 'start': 5538},
 {'end': 5547, 'id': 13, 'label': 'jobSkill', 'start': 5542},
 {'end': 5642, 'id': 14, 'label': 'jobSkill', 'start': 5632},
 {'end': 5699, 'id': 15, 'label': 'jobSkill', 'start': 5685},
 {'end': 5760, 'id': 16, 'label': 'jobSkill', 'start': 5743},
 {'end': 5779, 'id': 17, 'l

In [14]:
for document in data['documents']:
    del document['id']
    for ann in document['annotations']:
        del ann['id']

In [6]:
data['documents'][0]

{'annotations': [{'end': 13, 'label': 'jobTitle', 'start': 3},
  {'end': 31, 'label': 'jobSkill', 'start': 13},
  {'end': 501, 'label': 'jobSkill', 'start': 494},
  {'end': 1023, 'label': 'jobSkill', 'start': 1014},
  {'end': 1110, 'label': 'jobSkill', 'start': 1084},
  {'end': 1136, 'label': 'jobSkill', 'start': 1131},
  {'end': 1334, 'label': 'jobTitle', 'start': 1317},
  {'end': 1542, 'label': 'softSkill', 'start': 1530},
  {'end': 3350, 'label': 'jobTitle', 'start': 3325},
  {'end': 5508, 'label': 'jobSkill', 'start': 5498},
  {'end': 5536, 'label': 'jobSkill', 'start': 5528},
  {'end': 5540, 'label': 'jobSkill', 'start': 5538},
  {'end': 5547, 'label': 'jobSkill', 'start': 5542},
  {'end': 5642, 'label': 'jobSkill', 'start': 5632},
  {'end': 5699, 'label': 'jobSkill', 'start': 5685},
  {'end': 5760, 'label': 'jobSkill', 'start': 5743},
  {'end': 5779, 'label': 'jobSkill', 'start': 5762},
  {'end': 5857, 'label': 'jobSkill', 'start': 5851},
  {'end': 5902, 'label': 'softSkill', 'st

In [15]:
training_data = []

for document in data['documents']:
    if len(document['content']) < 1:
        print(f'missing contents', {document['content']},{document['annotations'][0]['label']})
        continue
    else:
        text = document['content']
        entities = document['annotations']
        training_data.append({'text':text,'entities':entities})

In [ ]:
nlp = spacy.load("en_core_web_sm") 
doc_bin = DocBin()

In [17]:
doc_count = len(training_data)
for d in training_data:
    label_count = len(d['entities'])
    print(label_count)

29
18
56
30
16
53
53
20
22
15
35
13
17
44
65
48
22
48
28
42
28
24
23
29
44
11
59
22
44
32
5
41
21
60
16
26
20
19
18
23
9
31
13
20
6
19
43
28
58
74
9
35
33
42
42
12
32
20
16
51
43
27
29
22
83
24
19
13
16
34
15
15
30
27
10
33
24
22
16
16
18
14
25
10
21
15
11
31
6
29
27
5
58
39
3
22
37
36
66
41
32
22
25
23
28
31
27
39
43
42
38
38
44
46
35
38
43
41
14
31
24
50
59
32
38
16
36
30
7
29
41
48
34
29
43
66
38
30
14
51
21
18
15
44
45
32
23
40
67
34
33
7
22
6
21
91
116
79
108
75
35
30
10
58
26
72
35
52
16
17
32
35
28
50
47
20
22
24
30
14
42
14
29
60
38
19
11
9
6
6
21
124
104
73
85
39
73
64
48
50
24
63
37
47
94
53
85
72
78
68
179
109
51
26
135
24
73
137
43
67
113
44
65
62
49
97
59
67
43
57
55
49
65
25
77
53
71
82
108
39
74
35
83
140
42
24
46
82
50
87
70
69
23
21
33
51
37
57
37
67
38
14
72
36
82
49
151
36
67
46
23
82
39
62
66
19
52
11
39
37
43
56
13
39
75
28
113
32
37
24
33
29
50
35
25
33
19
68
50
60
65
50
71
42
24
40
8
62
30
32
66
96
47
36
42
44
35
36
36
81
59
22
36
37
46
48
39
33
53
65
59
42
14
49

In [18]:
for d in tqdm(training_data):
    text = d['text']
    labels = d['entities']
    doc = nlp.make_doc(text)
    entities = []

    for l in labels:
        start = l['start']
        if text[start] == ' ':
            start +=1
        end = l['end']
        if text[end] == ' ':
            end -=1
        label = l['label']

        span = doc.char_span(start_idx=start, end_idx=end,label=label,alignment_mode='expand')
        entities.append(span)

100%|██████████| 976/976 [00:04<00:00, 232.85it/s]


In [19]:
doc.set_ents(entities)
doc_bin.add(doc)
doc_bin.to_disk("train.spacy")

In [21]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --force

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [22]:
! python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    505.50    0.00    0.00    0.00    0.00
^C


In [1]:
nlp_trained = spacy.load("model-best")

NameError: name 'spacy' is not defined

In [ ]:
with open('test.txt', 'r') as f:
    text = f.read()
doc = text


In [47]:
from IPython.display import HTML, display
html = displacy.render(doc, style="ent", jupyter=False)
display(HTML(f'<span class="tex2jax_ignore">{html}</span>'))

Data generation operation failed
 error details: AOAI resource authorization failed:
  Response: Forbidden, message: 
  {"error":
  {"code":"AuthorizationFailed",
  "message":
    "The client '007a9b5a-127f-4ae3-956a-6e0f29ec89ae' with object id '007a9b5a-127f-4ae3-956a-6e0f29ec89ae' does not have authorization to perform action 'Microsoft.CognitiveServices/accounts/listKeys/action' over scope '/subscriptions/aab34b8c-b2d8-42f1-af9b-72404d4d2e69/resourceGroups/cvbutbetter/providers/Microsoft.CognitiveServices/accounts/labellm' or the scope is invalid. If access was recently granted, please refresh your credentials."}} 